<a href="https://colab.research.google.com/github/AliaumeL/AtomicHilbert/blob/main/atomic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Self, TypeVar, Generic, Callable, List
from itertools import permutations, combinations
from operator import itemgetter
from fractions import Fraction
from abc import ABC, abstractmethod
from copy import deepcopy

In [2]:
class Atom:

    # The method to check whether two tuples of atoms are in the same S orbit.
    # The set S and the pair of tuples are input as a list.
    @abstractmethod
    def in_same_orbit(S,first_list,second_list): pass

    # The method to compute a set of representatives of S orbits of ordered non-repeating n-tuples of atoms.
    @abstractmethod
    def rep_of_orbits(S,n): pass

# The abstract class of effectively oligomorphic atoms with an effective order as a part of its structure.
class Atom_with_order(Atom):

    def __lt__(self,other): pass

    # The effective order
    @abstractmethod
    def is_less_than(self,other): pass

#todos
# 1. Operations to build new structures of atoms from old one:
#    a. Product
#    b. Lexicographic product
#
# 2. Specific instances of structures of atoms:
#    a. Equality
#    b. Ordered
#    c. Dense tree
#    d. Random graph
#    e. Finite sets with trivial automorphism group
#
# 3. Symbolic instantiation.
#    That is, instead of instantiating concretely, we specify the relations between the atoms.
#    There are two things to check:
#    namely if the specific is consistent, i.e. it has a solution, and if the solution is unique upto orbits.
#    This should be implemented at some point but maybe not immediatly.
#    We should also have a differnt print for these.
#
#    This is not difficult theoretically.
#    We just have to compute set of representatives and check if exactly one of them satisfy all the specifications

# 4. Define homogeneous atoms as taking a class and relations on them as input. Write same_orbit abstractly.
#    Put rep as abstract class
#

In [3]:
carrier = TypeVar('carrier')

class Relation(Generic[carrier]):

    def __init__(self,arity:int, rel : Callable[[list[carrier]],bool]):
        self.arity = arity
        self.rel   = rel

    def eval(self,input:list[carrier]) -> bool:
        if len(input) != self.arity :
            raise ValueError("Arity of relation doesn't match length of the input list")
        else:
            return self.relation(input)

In [4]:
def from_indices(indices : list[int], input_list : list[carrier]) -> list[carrier]:
    answer = []
    for index in indices:
        answer.append(input_list[index])
    return answer

In [5]:
class Homogeneous(Generic[carrier]):

    def __init__(self,name : carrier):
        self.name = name

    def __eq__(self:Self, other:Self):
        return (self.name == other.name)

    def equality_rel(input_list : list[carrier]):
        if len(input_list) != 2:
            raise ValueError("equality_rel inputs a list of length 2")
        else:
            return (input_list[0] == input_list[1])

    def __str__(self) -> str:
        return str(self.name)

    relations = [Relation(2,equality_rel)]

    @staticmethod
    def in_same_equiv_orbit(first_list : list[carrier], second_list : list) -> bool:
        if len(first_list) != len(second_list):
            return False
        else:
            answer = True
            n = len(first_list)
            for relation in __class__.relations:
                for pre_indices in list(permutations(range(n),relation.arity)):
                    indices = list(pre_indices)
                    if (relation.rel(from_indices(indices,first_list)) !=
                        relation.rel(from_indices(indices,second_list))):
                        answer = False
            return answer

    @staticmethod
    def in_same_orbit(support: list[carrier],first_list : list[carrier], second_list : list) -> bool:
        return __class__.in_same_equiv_orbit(support + first_list,support + second_list)

    @staticmethod
    @abstractmethod
    def rep_of_orbits(S,n): pass

In [6]:
class Equality_atom(Homogeneous[str]):

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

    @staticmethod
    def rep_of_orbits(support : list[Self], n : int):
        list1 : list[Self] = [Equality_atom("a"+str(k)) for k in range(1,(n + len(support) + 1))]
        list2 : list[Self] =  [a for a in list1 if a not in support]
        n_new_atoms = list2[:n]
        place_for_supp = []
        for k in range(max(n,len(support))+1):
            place_for_supp += list(permutations(range(n), k))
        answer = []

        for places in place_for_supp:
          for from_support in list(permutations(support,len(places))):
            to_add = deepcopy(n_new_atoms)
            for i in range(len(places)):
              to_add[places[i]] = from_support[i]
            if to_add not in answer:
              answer.append(to_add)
        return answer

# the above code is highly inefficient
# better to have method for S-orbits inside (A\S)^(n) and use it for inside S-orbits inside A^(n)
# Same for sets. This would be more efficient and anyway for ordered we need for sets

In [7]:
a : Equality_atom = Equality_atom("a")
b : Equality_atom = Equality_atom("b")
c : Equality_atom = Equality_atom("c")
len(Equality_atom.rep_of_orbits([a,b,c],1))

4

In [11]:
class DLO_atom(Homogeneous[Fraction]):

  def __str__(self):
    return str(self.name)

  def __repr__(self):
        return self.name

  def __lt__(self,other):
    return (self.name < other.name)

  @staticmethod
  def order(input_list:list[Fraction]):
    if len(input_list) != 2:
      raise ValueError("order inputs a list of length 2")
    else:
      return (input_list[0] < input_list[1])

  relations = Homogeneous.relations + [Relation(2,order)]

[<__main__.Relation object at 0x7d56c5cf9350>, <__main__.Relation object at 0x7d56b10cc810>]


In [9]:
a = DLO_atom(1)
b = DLO_atom(2)
c = DLO_atom(3)
DLO_atom.in_same_orbit([b],[a],[c])

True